In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from catboost import CatBoostRegressor
from tqdm import tqdm

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "./input"]).decode("utf8"))

train_df = pd.read_csv('./input/train_2016_v2.csv', parse_dates=['transactiondate'], low_memory=False)
test_df = pd.read_csv('./input/sample_submission.csv', low_memory=False)
properties = pd.read_csv('./input/properties_2016.csv', low_memory=False)
# field is named differently in submission
test_df['parcelid'] = test_df['ParcelId']

# similar to the1owl
def add_date_features(df):
    df["transaction_year"] = df["transactiondate"].dt.year
    df["transaction_month"] = df["transactiondate"].dt.month
    df["transaction_day"] = df["transactiondate"].dt.day
    df["transaction_quarter"] = df["transactiondate"].dt.quarter
    df.drop(["transactiondate"], inplace=True, axis=1)
    return df
train_df = add_date_features(train_df)
#test_df = add_date_features(test_df)

properties_2016.csv
sample_submission.csv
train_2016_v2.csv
zillow_data_dictionary.xlsx



In [2]:
def add_pro_features(df_train):
    
    
    df_train['N-life'] = 2018 - df_train['yearbuilt']

    #error in calculation of the finished living area of home
    df_train['N-LivingAreaError'] = df_train['calculatedfinishedsquarefeet']/df_train['finishedsquarefeet12']

    #proportion of living area
    df_train['N-LivingAreaProp'] = df_train['calculatedfinishedsquarefeet']/df_train['lotsizesquarefeet']
    df_train['N-LivingAreaProp2'] = df_train['finishedsquarefeet12']/df_train['finishedsquarefeet15']

    #Amout of extra space
    df_train['N-ExtraSpace'] = df_train['lotsizesquarefeet'] - df_train['calculatedfinishedsquarefeet'] 
    df_train['N-ExtraSpace-2'] = df_train['finishedsquarefeet15'] - df_train['finishedsquarefeet12'] 

    #Total number of rooms
    df_train['N-TotalRooms'] = df_train['bathroomcnt']*df_train['bedroomcnt']

    #Average room size
    df_train['N-AvRoomSize'] = df_train['calculatedfinishedsquarefeet']/df_train['roomcnt'] 

    # Number of Extra rooms
    df_train['N-ExtraRooms'] = df_train['roomcnt'] - df_train['N-TotalRooms'] 

    #Ratio of the built structure value to land area
    df_train['N-ValueProp'] = df_train['structuretaxvaluedollarcnt']/df_train['landtaxvaluedollarcnt']

    #Does property have a garage, pool or hot tub and AC?
    df_train['N-GarPoolAC'] = ((df_train['garagecarcnt']>0) & (df_train['pooltypeid10']>0) & (df_train['airconditioningtypeid']!=5))*1 

    df_train["N-location"] = df_train["latitude"] + df_train["longitude"]
    df_train["N-location-2"] = df_train["latitude"]*df_train["longitude"]
    df_train["N-location-2round"] = df_train["N-location-2"].round(-4)

    df_train["N-latitude-round"] = df_train["latitude"].round(-4)
    df_train["N-longitude-round"] = df_train["longitude"].round(-4)

    #Ratio of tax of property over parcel
    df_train['N-ValueRatio'] = df_train['taxvaluedollarcnt']/df_train['taxamount']

    #TotalTaxScore
    df_train['N-TaxScore'] = df_train['taxvaluedollarcnt']*df_train['taxamount']

    #polnomials of tax delinquency year
    df_train["N-taxdelinquencyyear-2"] = df_train["taxdelinquencyyear"] ** 2
    df_train["N-taxdelinquencyyear-3"] = df_train["taxdelinquencyyear"] ** 3

    #Length of time since unpaid taxes
    df_train['N-life'] = 2018 - df_train['taxdelinquencyyear']

    #Number of properties in the zip
    zip_count = df_train['regionidzip'].value_counts().to_dict()
    df_train['N-zip_count'] = df_train['regionidzip'].map(zip_count)

    #Number of properties in the city
    city_count = df_train['regionidcity'].value_counts().to_dict()
    df_train['N-city_count'] = df_train['regionidcity'].map(city_count)

    #Number of properties in the city
    region_count = df_train['regionidcounty'].value_counts().to_dict()
    df_train['N-county_count'] = df_train['regionidcounty'].map(city_count)

    #Indicator whether it has AC or not
    df_train['N-ACInd'] = (df_train['airconditioningtypeid']!=5)*1

    #Indicator whether it has Heating or not 
    df_train['N-HeatInd'] = (df_train['heatingorsystemtypeid']!=13)*1

    #There's 25 different property uses - let's compress them down to 4 categories
    df_train['N-PropType'] = df_train.propertylandusetypeid.replace({31 : "Mixed", 46 : "Other", 47 : "Mixed", 246 : "Mixed", 247 : "Mixed", 248 : "Mixed", 260 : "Home", 261 : "Home", 262 : "Home", 263 : "Home", 264 : "Home", 265 : "Home", 266 : "Home", 267 : "Home", 268 : "Home", 269 : "Not Built", 270 : "Home", 271 : "Home", 273 : "Home", 274 : "Other", 275 : "Home", 276 : "Home", 279 : "Home", 290 : "Not Built", 291 : "Not Built" })

    #polnomials of the variable
    df_train["N-structuretaxvaluedollarcnt-2"] = df_train["structuretaxvaluedollarcnt"] ** 2
    df_train["N-structuretaxvaluedollarcnt-3"] = df_train["structuretaxvaluedollarcnt"] ** 3

    #Average structuretaxvaluedollarcnt by city
    group = df_train.groupby('regionidcity')['structuretaxvaluedollarcnt'].aggregate('mean').to_dict()
    df_train['N-Avg-structuretaxvaluedollarcnt'] = df_train['regionidcity'].map(group)

    #Deviation away from average
    df_train['N-Dev-structuretaxvaluedollarcnt'] = abs((df_train['structuretaxvaluedollarcnt'] - df_train['N-Avg-structuretaxvaluedollarcnt']))/df_train['N-Avg-structuretaxvaluedollarcnt']
    
    return df_train

properties = add_pro_features(properties)

In [3]:
train_df = train_df.merge(properties, how='left', on='parcelid')
# train_df= train_df[ train_df.logerror > -0.4]
# train_df= train_df[ train_df.logerror < 0.419]
test_df = test_df.merge(properties, how='left', on='parcelid')
print("Train: ", train_df.shape)
print("Test: ", test_df.shape)

('Train: ', (90275, 93))
('Test: ', (2985217, 95))


In [4]:
train_df.head()

,parcelid,logerror,transaction_year,transaction_month,transaction_day,transaction_quarter,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,...,N-zip_count,N-city_count,N-county_count,N-ACInd,N-HeatInd,N-PropType,N-structuretaxvaluedollarcnt-2,N-structuretaxvaluedollarcnt-3,N-Avg-structuretaxvaluedollarcnt,N-Dev-structuretaxvaluedollarcnt
0,11016594,0.0276,2016,1,1,1,1.0,NaN,NaN,2.0,...,14719.0,670925.0,NaN,1,1,Home,1.506854e+10,1.849724e+15,182727.848429,0.328214
1,14366692,-0.1684,2016,1,1,1,NaN,NaN,NaN,3.5,...,17682.0,22312.0,NaN,1,1,Home,1.200331e+11,4.158644e+16,150923.968820,1.295580
2,12098116,-0.0040,2016,1,1,1,1.0,NaN,NaN,3.0,...,4422.0,33795.0,NaN,1,1,Home,3.843256e+09,2.382588e+14,179421.366426,0.654478
3,12643413,0.0218,2016,1,2,1,1.0,NaN,NaN,2.0,...,7293.0,670925.0,NaN,1,1,Home,2.941842e+10,5.045789e+15,182727.848429,0.061347
4,14432541,-0.0050,2016,1,2,1,NaN,NaN,NaN,2.5,...,9875.0,9746.0,NaN,1,1,Home,2.875534e+10,4.876158e+15,220682.365094,0.231592
